<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 2s 0us/step


In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [6]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [9]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [11]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 1154s 12s/step - loss: 0.3555 - accuracy: 0.8880 - val_loss: 0.3255 - val_accuracy: 0.8973
Epoch 2/5
94/94 [==============================] - 17s 185ms/step - loss: 0.1810 - accuracy: 0.9447 - val_loss: 0.3012 - val_accuracy: 0.8917
Epoch 3/5
94/94 [==============================] - 17s 183ms/step - loss: 0.1026 - accuracy: 0.9755 - val_loss: 0.2754 - val_accuracy: 0.9051
Epoch 4/5
94/94 [==============================] - 16s 167ms/step - loss: 0.0630 - accuracy: 0.9920 - val_loss: 0.2670 - val_accuracy: 0.9107
Epoch 5/5
94/94 [==============================] - 16s 170ms/step - loss: 0.0386 - accuracy: 0.9970 - val_loss: 0.2141 - val_accuracy: 0.9275


In [12]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/MobileNet.h5')

In [13]:

model.evaluate(val_generator)

29/29 [==============================] - 4s 139ms/step - loss: 0.2091 - accuracy: 0.9294


[0.20905889570713043, 0.929424524307251]

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
# New_model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/MobileNet.h5')

# Show the model architecture
New_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
New_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=2
)


Epoch 1/2
94/94 [==============================] - 20s 204ms/step - loss: 0.1002 - accuracy: 0.9816 - val_loss: 0.4772 - val_accuracy: 0.8471
Epoch 2/2
94/94 [==============================] - 18s 189ms/step - loss: 0.0754 - accuracy: 0.9866 - val_loss: 0.4479 - val_accuracy: 0.8460


In [ ]:
# Save the trained model
New_model.save('/content/drive/MyDrive/Dataset/MobileNet01.h5')